XGBoost

In [16]:
import xgboost as xgb
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
import feature_engineering
from sklearn.model_selection import train_test_split
from display_data import import_data

I want to use import_data function from another directory, and make the data ready for the model. LightGBM accepts categorical values, but they have to be encoded as no-negative integers.


In [17]:
#We dont want our model to care about the id of the house or the seller
data, data_test = import_data()
Y = data.price
print(Y)
for column in data.columns:
    column_type = data[column].dtype
    if column_type == 'object':
        break
    data[column] = data[column].replace(np.NaN, data[column].mean())
#print(data_test)
#Features
radius = False
district_mean_price = True
# Add radius 
if(radius):
    data = feature_engineering.add_radius(data)
    data_test = feature_engineering.add_radius(data_test)
    coordinates = ['latitude', 'longitude']
    data = data.drop(columns = coordinates)
    data_test = data_test.drop(columns = coordinates)
if(district_mean_price):
    data = feature_engineering.group_by_feature_and_price(data,'district')
    data_test = feature_engineering.group_by_feature_and_price(data_test,'district',data_train=data)
#print(data_test)
#Remove catdata
data = data.drop(columns=['price', 'id','seller'])
categorical_data = ['layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating']
data = data.drop(columns = categorical_data)
data_test = data_test.drop(columns=['id','seller'])
data_test = data_test.drop(columns = categorical_data)

X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)

0         7139520.0
1        10500000.0
2         9019650.0
3        10500000.0
4        13900000.0
            ...    
23280    13300000.0
23281    15854300.0
23282    19800000.0
23283    29999000.0
23284    10950000.0
Name: price, Length: 23285, dtype: float64


In [18]:


def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

#data.info()
#print(len(Y))
#print(data.corr())

XGB_model = xgb.XGBRegressor(n_estimators = 10000)

XGB_model.fit(X_train,y_train)
prediction = XGB_model.predict(X_test)

##############




#lgb.plot_importance(lightGBM_model)

count = 0
for i in prediction:
    if i < 0:
        prediction[count] = prediction.mean()
    count += 1

rmsle = root_mean_squared_log_error(y_test,prediction)
print("first run", rmsle)


"""
lightGBM_model = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,8,9,10),
    num_leaves=256,
    max_depth=8,
    random_state=42,
    metric='rmsle',
    num_iterations=2000,
    learning_rate=0.09,
    bagging_freq = 5,
    bagging_fraction = 0.8)
"""

first run 0.19084295853893113


"\nlightGBM_model = lgb.LGBMRegressor(\n    categorical_feature=(0,1,2,8,9,10),\n    num_leaves=256,\n    max_depth=8,\n    random_state=42,\n    metric='rmsle',\n    num_iterations=2000,\n    learning_rate=0.09,\n    bagging_freq = 5,\n    bagging_fraction = 0.8)\n"

Runs
Only numeric, no feature engineered: 0.19744699138034857

Radius added: 0.20967165360778273, Why worse

Distric_mean: 0.19084295853893113